In [166]:
##import packages

import requests
import json
import pandas as pd
import numpy as np
from decimal import Decimal
import itertools as it
import plotly.express as px
import plotly.graph_objects as go


In [167]:
pip install nbformat>=4.2.0

Note: you may need to restart the kernel to use updated packages.


In [168]:
##Request fixtures from FPL API

url = f"https://fantasy.premierleague.com/api/fixtures/"
response = requests.get(url)
json_data = response.json()

Fixtures_Dim = json_data

Fixtures_Dim = pd.json_normalize(Fixtures_Dim)


Fixtures_Dim_Future=Fixtures_Dim[Fixtures_Dim['finished']==False]

### Used as variable to return only the data for completed gameweeks

UpcomingWeek = Fixtures_Dim_Future['event'].min().astype(int)
PrevWeek = UpcomingWeek-1

### Variable for number of gameweeks to go back to calculate averages from

NoTrailingWeeks = 5

NoForecastWeeks = 4

TeamID = 1769272

print(UpcomingWeek)
print(PrevWeek)


22
21


In [169]:
###Request Team Data from FPL API

url = f"https://fantasy.premierleague.com/api/bootstrap-static"
response = requests.get(url)
json_data = response.json()

teams_dim = json_data['teams']

Teams_Dim = pd.json_normalize(teams_dim)



In [170]:
###Duplicates of Fixtures for Home and Away, so that this can be merged with elements

Fixtures_Away1 = Fixtures_Dim_Future
Fixtures_Away1.head()

Fixtures_Home1 = Fixtures_Dim_Future
Fixtures_Home1.head()

FixturesConcat = pd.concat([Fixtures_Away1,Fixtures_Home1], keys=["Away", "Home"])

FixturesConcat.reset_index(inplace=True)

FixturesConcat['TeamID'] = FixturesConcat.apply(
    lambda row: row['team_a'] if row['level_0'] == 'Away' else row['team_h'], 
    axis=1
)

FixturesConcat['OppTeam2'] = FixturesConcat.apply(
    lambda row: row['team_a'] if row['level_0'] == 'Home' else row['team_h'], 
    axis=1
)

In [171]:
### Gameweek elements data from FPL API

player_data = []

for gw1 in range(1,UpcomingWeek):
    url = f"https://fantasy.premierleague.com/api/event/{gw1}/live/"
    
    response = requests.get(url)
    json_data = response.json()
    print(url)

    for gw in json_data['elements']:
        #i1 = gw['id']
        playerid=gw['id']
        minutes=gw['stats']['minutes']
        goals_scored=gw['stats']['goals_scored']
        assists=gw['stats']['assists']
        clean_sheets=gw['stats']['clean_sheets']
        goals_conceded=gw['stats']['goals_conceded']
        own_goals=gw['stats']['own_goals']
        penalties_saved=gw['stats']['penalties_saved']
        penalties_missed=gw['stats']['penalties_missed']
        yellow_cards=gw['stats']['yellow_cards']
        red_cards=gw['stats']['red_cards']
        saves=gw['stats']['saves']
        bonus=gw['stats']['bonus']
        bps=gw['stats']['bps']
        influence=gw['stats']['influence']
        creativity=gw['stats']['creativity']
        threat=gw['stats']['threat']
        ict_index=gw['stats']['ict_index']
        starts=gw['stats']['starts']
        expected_goals=gw['stats']['expected_goals']
        expected_assists=gw['stats']['expected_assists']
        expected_goal_involvements=gw['stats']['expected_goal_involvements']
        expected_goals_conceded=gw['stats']['expected_goals_conceded']
        total_points=gw['stats']['total_points']
        if gw['explain']: 
            fixtureid = gw['explain'][0]['fixture'] 
        else: 
            fixtureid = None

        player_data.append({
            'playerid': playerid,
            'minutes': minutes,
            'goals_scored': goals_scored,
            'assists': assists,
            'clean_sheets': clean_sheets,
            'goals_conceded': goals_conceded,
            'threat': threat,
            'expected_goals': expected_goals,
            'expected_assists': expected_assists,
            'expected_goal_involvements': expected_goal_involvements,
            'expected_goals_conceded': expected_goals_conceded,
            'fixtureid': fixtureid  


            })

    GWElementsDF = pd.DataFrame(player_data)
GWElementsDF.head(5)

GWElementsDF1 = pd.merge(GWElementsDF, Fixtures_Dim,left_on='fixtureid', right_on='id', how='outer')

GWElementsDF1.to_csv('FPL_Event_By_Week.csv')
GWElementsDF1.head()


https://fantasy.premierleague.com/api/event/1/live/
https://fantasy.premierleague.com/api/event/2/live/
https://fantasy.premierleague.com/api/event/3/live/
https://fantasy.premierleague.com/api/event/4/live/
https://fantasy.premierleague.com/api/event/5/live/
https://fantasy.premierleague.com/api/event/6/live/
https://fantasy.premierleague.com/api/event/7/live/
https://fantasy.premierleague.com/api/event/8/live/
https://fantasy.premierleague.com/api/event/9/live/
https://fantasy.premierleague.com/api/event/10/live/
https://fantasy.premierleague.com/api/event/11/live/
https://fantasy.premierleague.com/api/event/12/live/
https://fantasy.premierleague.com/api/event/13/live/
https://fantasy.premierleague.com/api/event/14/live/
https://fantasy.premierleague.com/api/event/15/live/
https://fantasy.premierleague.com/api/event/16/live/
https://fantasy.premierleague.com/api/event/17/live/
https://fantasy.premierleague.com/api/event/18/live/
https://fantasy.premierleague.com/api/event/19/live/
ht

,playerid,minutes_x,goals_scored,assists,clean_sheets,goals_conceded,threat,expected_goals,expected_assists,expected_goal_involvements,...,provisional_start_time,started,team_a,team_a_score,team_h,team_h_score,stats,team_h_difficulty,team_a_difficulty,pulse_id
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,...,False,True,20.0,0.0,1.0,2.0,"[{'identifier': 'goals_scored', 'a': [], 'h': ...",2.0,5.0,115828.0
1,2.0,5.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,...,False,True,20.0,0.0,1.0,2.0,"[{'identifier': 'goals_scored', 'a': [], 'h': ...",2.0,5.0,115828.0
2,3.0,90.0,0.0,0.0,1.0,0.0,8.0,0.12,0.02,0.14,...,False,True,20.0,0.0,1.0,2.0,"[{'identifier': 'goals_scored', 'a': [], 'h': ...",2.0,5.0,115828.0
3,4.0,90.0,1.0,1.0,1.0,0.0,46.0,0.45,0.04,0.49,...,False,True,20.0,0.0,1.0,2.0,"[{'identifier': 'goals_scored', 'a': [], 'h': ...",2.0,5.0,115828.0
4,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,...,False,True,20.0,0.0,1.0,2.0,"[{'identifier': 'goals_scored', 'a': [], 'h': ...",2.0,5.0,115828.0


In [172]:
###Standard Element Data from FPL API

url = f"https://fantasy.premierleague.com/api/bootstrap-static"
response = requests.get(url)
json_data = response.json()

elements_dim = json_data['elements']

Elements_Dim1 = pd.json_normalize(elements_dim)

Elements_Dim1.to_csv('elementsdim.csv')

Elements_Dim1 = pd.merge(GWElementsDF1, Elements_Dim1,left_on='playerid', right_on='id', how='outer',suffixes=('.gw', '.info'))

Elements_Dim1['ExcludeDef'] = Elements_Dim1.apply(
    lambda row: 'Y' if (row['element_type'] == '1' or row['element_type'] == '2') and row['minutes_x'] > 59 else 'N', 
    axis=1
)

Elements_Dim1.to_csv('elements4wk.csv')

In [173]:
###Arbitrary team weightings calculated based on average of last season's odds

TeamWeightings = {
    "Weighting": [
        1.699153895, 1.68275948, 1.639154108, 1.626396476, 1.605876872, 1.542304678,
        1.526424188, 1.50219836, 1.499125082, 1.489893392, 1.44492154, 1.424635514,
        1.365324148, 1.288771283, 1.266010528, 1.251813146, 1.18226904, 1.7, 1.7, 1.7
    ],
    "Teams.id": [
        3, 7, 16, 20, 4, 9, 8, 2, 15, 19, 5, 14, 6, 12, 18, 1, 13, 10, 11, 17
    ],
    "Teams.name2": [
        "Bournemouth", "Crystal Palace", "Nott'm Forest", "Wolves", "Brentford", "Fulham",
        "Everton", "Aston Villa", "Newcastle", "West Ham", "Brighton", "Man Utd", "Chelsea",
        "Liverpool", "Spurs", "Arsenal", "Man City", "Ipswich", "Leicester", "Southampton"
    ]
    ,
    "Teams.name": [
        "Bour", "CryP", "For't", "Wolv", "Bfrd", "Ful",
        "Ev", "Villa", "Newc", "WHam", "Bri", "ManU", "Chel",
        "LPool", "Spurs", "Ars", "ManC", "Ips", "Leic", "Soton"
    ]
        ,
    "Teams.name1": [
        "b", "c", "F", "W", "B", "F",
        "E", "V", "N", "W", "B", "M", "Ch",
        "L", "S", "A", "M", "I", "L", "S"
    ]
}

TeamWeightings = pd.DataFrame(TeamWeightings)

Elements_Dim1['OppTeam'] = Elements_Dim1.apply(lambda row: row['team_h'] if row['team_h'] == row['team'] else row['team_a'], axis=1)

Elements_Dim2 = pd.merge(Elements_Dim1, TeamWeightings,left_on='OppTeam', right_on='Teams.id', how='outer')

Elements_Dim2['XGC_Weighted'] = Elements_Dim2['Weighting'].astype(float)*Elements_Dim2['expected_goals_conceded.gw'].astype(float)

Elements_Dim2['XGI_Weighted'] = Elements_Dim2['expected_goal_involvements.gw'].astype(float)/Elements_Dim2['Weighting'].astype(float)



In [174]:
###Filtered to only include the previous 4 weeks' data before it is grouped to return total sum for each team 

Elements_Dim2Filtered = Elements_Dim2[Elements_Dim2['event']>PrevWeek-NoTrailingWeeks]

Elements_Dim2Grouped1 = Elements_Dim2Filtered.groupby(['OppTeam','Teams.name', 'event']).agg(
    XGI_Weighted=('XGI_Weighted', 'sum'),
    XGC_Weighted=('XGC_Weighted', 'max'),
    Minutes=('minutes_x', 'sum')
).reset_index()



##Average of the above taken for each team
Elements_Dim2Grouped2 = Elements_Dim2Grouped1.groupby(['OppTeam','Teams.name']).agg(
    XGI_Weighted=('XGI_Weighted', 'mean'),
    XGC_Weighted=('XGC_Weighted', 'mean'),
    Minutes=('Minutes', 'mean')
).reset_index()



Elements_Dim2Grouped2.head()

Elements_Dim2Grouped2.to_csv('TeamProfiles.csv')

In [175]:
##Previous 4 Gameweeks this dataframe is returning the averages for the selected players
Elements_Dim1 = Elements_Dim1[Elements_Dim1['event']<UpcomingWeek]

GWElementsDFLast4Weeks = Elements_Dim1[Elements_Dim1['event']>UpcomingWeek-NoTrailingWeeks-1]

GWElementsDFLast4Weeks = GWElementsDFLast4Weeks[['minutes_x','playerid','web_name','element_type','team','now_cost','expected_goals.gw','expected_assists.gw','expected_goal_involvements.gw','expected_goals_conceded.gw']]

GWElementsDFLast4Weeks['expected_goals.gw'] = GWElementsDFLast4Weeks['expected_goals.gw'].astype(float)
GWElementsDFLast4Weeks['expected_assists.gw'] = GWElementsDFLast4Weeks['expected_assists.gw'].astype(float)
GWElementsDFLast4Weeks['expected_goal_involvements.gw'] = GWElementsDFLast4Weeks['expected_goal_involvements.gw'].astype(float)
GWElementsDFLast4Weeks['expected_goals_conceded.gw'] = GWElementsDFLast4Weeks['expected_goals_conceded.gw'].astype(float)

GWElementsDFLast4Weeks = GWElementsDFLast4Weeks.groupby(['playerid','web_name','element_type','now_cost','team'], as_index=False)[['minutes_x','expected_goals.gw','expected_assists.gw','expected_goal_involvements.gw','expected_goals_conceded.gw']].mean(numeric_only=True)

GWElementsDFLast4Weeks['ExcludeDef'] = GWElementsDFLast4Weeks.apply(
    lambda row: 'Y' if (row['element_type'] == 1 or row['element_type'] == 2) and row['minutes_x']< 59 else 'N', 
    axis=1
)

grouped_dfLast4Weeks=GWElementsDFLast4Weeks[GWElementsDFLast4Weeks['ExcludeDef']!='Y']
grouped_dfLast4Weeks.head()

,playerid,web_name,element_type,now_cost,team,minutes_x,expected_goals.gw,expected_assists.gw,expected_goal_involvements.gw,expected_goals_conceded.gw,ExcludeDef
0,1.0,Fábio Vieira,3.0,54.0,1.0,0.0,0.000,0.000,0.000,0.000,N
1,2.0,G.Jesus,4.0,67.0,1.0,67.2,0.406,0.076,0.482,0.676,N
2,3.0,Gabriel,2.0,64.0,1.0,90.0,0.138,0.078,0.216,0.868,N
3,4.0,Havertz,4.0,78.0,1.0,47.4,0.472,0.020,0.492,0.494,N
5,6.0,J.Timber,2.0,56.0,1.0,71.0,0.008,0.042,0.050,0.556,N


In [176]:
### Returns combinations of historic XG stats from players multiplied by historic opposition team performance

OpportunitiesCalc = pd.merge(grouped_dfLast4Weeks, Elements_Dim2Grouped2,left_on='team', right_on='OppTeam', how='outer')

OpportunitiesCalc['AttackOpp2'] = OpportunitiesCalc['XGC_Weighted']*OpportunitiesCalc['expected_goal_involvements.gw']

OpportunitiesCalc['DefenceOpp2'] = OpportunitiesCalc['XGI_Weighted']*OpportunitiesCalc['expected_goal_involvements.gw']

OpportunitiesCalc.sort_values(by=['AttackOpp2'])


,playerid,web_name,element_type,now_cost,team,minutes_x,expected_goals.gw,expected_assists.gw,expected_goal_involvements.gw,expected_goals_conceded.gw,ExcludeDef,OppTeam,Teams.name,XGI_Weighted,XGC_Weighted,Minutes,AttackOpp2,DefenceOpp2
0,1.0,Fábio Vieira,3.0,54.0,1.0,0.0,0.000,0.000,0.000,0.000,N,1.0,Ars,0.946934,1.086574,379.615385,0.000000,0.000000
141,722.0,Adu-Adjei,4.0,45.0,3.0,0.0,0.000,0.000,0.000,0.000,N,3.0,Bour,0.546879,2.389010,379.461538,0.000000,0.000000
142,723.0,Sadi,3.0,45.0,3.0,0.0,0.000,0.000,0.000,0.000,N,3.0,Bour,0.546879,2.389010,379.461538,0.000000,0.000000
306,678.0,Bates,3.0,45.0,8.0,0.0,0.000,0.000,0.000,0.000,N,8.0,Ev,0.310717,2.283531,352.642857,0.000000,0.000000
304,614.0,Metcalfe,3.0,45.0,8.0,0.0,0.000,0.000,0.000,0.000,N,8.0,Ev,0.310717,2.283531,352.642857,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
376,364.0,Amad,3.0,56.0,14.0,87.6,0.396,0.264,0.660,1.562,N,14.0,ManU,0.681879,2.381991,349.214286,1.572114,0.450040
189,110.0,Wissa,4.0,62.0,4.0,86.8,0.712,0.042,0.754,1.326,N,4.0,Bfrd,0.649845,2.129393,352.857143,1.605562,0.489983
249,306.0,Vardy,4.0,55.0,11.0,72.0,0.512,0.112,0.624,1.142,N,11.0,Leic,0.375630,2.590800,352.428571,1.616659,0.234393
427,401.0,Isak,4.0,94.0,15.0,82.8,1.094,0.118,1.212,0.690,N,15.0,Newc,1.014951,1.355209,379.538462,1.642513,1.230121


In [177]:
url = f"https://fantasy.premierleague.com/api/entry/{TeamID}/event/{PrevWeek}/picks/"
response = requests.get(url)
json_data = response.json()

TeamSelection = json_data['picks']

TeamSelection = pd.json_normalize(TeamSelection)

TeamSelection1 = TeamSelection[['element','is_captain']]

TeamSelection1.head()

,element,is_captain
0,15,False
1,255,False
2,594,False
3,422,False
4,78,False


In [178]:
##### Returns Output 1 FixturesElementsAll
FixturesElementsAll1 = pd.merge(FixturesConcat, Elements_Dim2Grouped2,left_on='OppTeam2', right_on='OppTeam', how='outer')

FixturesElementsAll2 = pd.merge(grouped_dfLast4Weeks, FixturesElementsAll1,left_on='team', right_on='TeamID', how='outer')

FixturesElementsAll3 =  pd.merge(TeamSelection1, FixturesElementsAll2,left_on='element', right_on='playerid', how='outer')

FixturesElementsAll3['InCurrentTeam']= np.where(
    FixturesElementsAll3['element'].notnull(),
    1,
    0
)

FixturesElementsAll3['AttackOpp_pergame'] = FixturesElementsAll3['XGC_Weighted']*FixturesElementsAll3['expected_goal_involvements.gw']

FixturesElementsAll3['DefenceOpp_pergame'] = FixturesElementsAll3['XGI_Weighted']*FixturesElementsAll3['expected_goals_conceded.gw']

FixturesElementsAll3.to_csv('FixturesElementsAll.csv')

FixturesElementsAll3['xGIAvg_Player']=FixturesElementsAll3['expected_goal_involvements.gw'].astype(str)+'-'+FixturesElementsAll3['web_name']


FixturesElementsAll2Att=FixturesElementsAll3[FixturesElementsAll3['element_type']>2]

# Calculate total AttackOpp_pergame for each player
FixturesElementsAll2Att = FixturesElementsAll2Att[FixturesElementsAll2Att['event']<UpcomingWeek+10]
FixturesElementsAll2_totalsAtt = FixturesElementsAll2Att.groupby('web_name')['AttackOpp_pergame'].sum().reset_index()

# Sort players by total AttackOpp_pergame and select top 20
top_players = FixturesElementsAll2_totalsAtt.sort_values(by='AttackOpp_pergame', ascending=False).head(20)

# Filter original DataFrame to include only top 20 players
FixturesElementsAll2AttFiltered = FixturesElementsAll2Att[FixturesElementsAll3['web_name'].isin(top_players['web_name'])]



C:\Users\james\AppData\Local\Temp\ipykernel_32140\164541063.py:33: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [183]:
import plotly.graph_objects as go
import pandas as pd

# Assuming FixturesElementsAll2Viz is your DataFrame
# FixturesElementsAll2Viz = pd.DataFrame(...) # your data here

# Pivot the DataFrame using pivot_table with an aggregation function
heatmap_data = FixturesElementsAll2AttFiltered.pivot_table(
    index='xGIAvg_Player',
    columns='event',
    values='XGC_Weighted',
    aggfunc='mean'  # You can change this to an appropriate function like sum, max, etc.
)
text_data = FixturesElementsAll2AttFiltered.pivot_table(
    index='xGIAvg_Player',
    columns='event',
    values='Teams.name',
    aggfunc=lambda x: ' | '.join(x)  # Aggregates Team.name values
)

# Create the heatmap
fig = go.Figure(data=go.Heatmap(
    z=heatmap_data.values,
    x=heatmap_data.columns,
    y=heatmap_data.index,
    text=text_data.values,
    texttemplate="%{text}",
    textfont={"size":10},
    colorscale='reds'
))

# Display the figure
fig.show()



In [180]:
##### Returns Output 2 FixturesElementsAllGrouped for previous 4 weeks

FixturesElementsAllGrouped = FixturesElementsAll2[FixturesElementsAll2['event']>UpcomingWeek-NoForecastWeeks-1]

FixturesElementsAllGrouped = FixturesElementsAllGrouped[['playerid','web_name','element_type','team','now_cost','expected_goals.gw','expected_assists.gw','expected_goal_involvements.gw','expected_goals_conceded.gw','XGI_Weighted','XGC_Weighted','AttackOpp_pergame','DefenceOpp_pergame']]

FixturesElementsAllGrouped = FixturesElementsAllGrouped.groupby(['playerid','web_name','element_type','now_cost','team'], as_index=False)[['expected_goals.gw','expected_assists.gw','expected_goal_involvements.gw','expected_goals_conceded.gw','XGI_Weighted','XGC_Weighted','AttackOpp_pergame','DefenceOpp_pergame']].mean(numeric_only=True)

FixturesElementsAllGrouped.to_csv('FixturesElementsAllGrouped.csv')

KeyError: "['AttackOpp_pergame', 'DefenceOpp_pergame'] not in index"

In [16]:
FixturesElementsAllGroupedwithTS = pd.merge(FixturesElementsAllGrouped, TeamSelection,left_on='playerid', right_on='element', how='outer')

FixturesElementsAllGroupedwithTS['IsInCurrentTeam'] = np.where(
    FixturesElementsAllGroupedwithTS['is_captain'].notnull(),
    'Y',
    'N'
)

In [17]:
FixturesElementsAllGroupedwithTS.head()

,playerid,web_name,element_type_x,now_cost,team,expected_goals.gw,expected_assists.gw,expected_goal_involvements.gw,expected_goals_conceded.gw,XGI_Weighted,XGC_Weighted,AttackOpp_pergame,DefenceOpp_pergame,element,position,multiplier,is_captain,is_vice_captain,element_type_y,IsInCurrentTeam
0,1.0,Fábio Vieira,3.0,54.0,1.0,0.000,0.000,0.000,0.000,0.610305,2.2884,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,N
1,2.0,G.Jesus,4.0,67.0,1.0,0.406,0.076,0.482,0.676,0.610305,2.2884,1.103009,0.412566,2.0,15.0,0.0,False,False,4.0,Y
2,3.0,Gabriel,2.0,64.0,1.0,0.138,0.078,0.216,0.868,0.610305,2.2884,0.494294,0.529745,NaN,NaN,NaN,NaN,NaN,NaN,N
3,4.0,Havertz,4.0,78.0,1.0,0.472,0.020,0.492,0.494,0.610305,2.2884,1.125893,0.301491,NaN,NaN,NaN,NaN,NaN,NaN,N
4,6.0,J.Timber,2.0,57.0,1.0,0.008,0.042,0.050,0.556,0.610305,2.2884,0.114420,0.339330,NaN,NaN,NaN,NaN,NaN,NaN,N


In [18]:
FixturesElementsAllGroupedwithTS.to_csv('FixturesElementsAllGroupedwithTS.csv')

In [ ]:
# ##Import the packages to carry out the optimisation problem

# import requests
# from sklearn.preprocessing import MinMaxScaler
# import warnings
# from pulp import *
# Small_Elements_df = FixturesElementsAllGroupedwithTS

In [20]:
# def AttackOptimisationproblem(Source,AttSpend,FreeTransfers):
#     Attack_Elements_df = Source.dropna(subset=['AttackOpp_pergame'])
#     Attack_Elements_df['AttackOpp_pergame'].astype(float)
#     Attack_Elements_df=Attack_Elements_df[Attack_Elements_df['element_type_x']!=1]
#     Attack_Elements_df=Attack_Elements_df[Attack_Elements_df['element_type_x']!=2]
#     ## Attack Variables
#     Players = list(Attack_Elements_df['web_name'])
#     Cost = dict(zip(Players, Attack_Elements_df['now_cost']))
#     Score = dict(zip(Players, Attack_Elements_df['AttackOpp_pergame'].astype(float)))
#     positions = dict(zip(Players, Attack_Elements_df['element_type_x']))
#     CurrentSelection = dict(zip(Players, Attack_Elements_df['IsInCurrentTeam']))
    
#     player_vars = LpVariable.dicts("Player", Players, lowBound=0, upBound=1, cat='Integer')

#     ##Attack Constraints and problem solve

#     total_score = LpProblem("Fantasy_Points_Problem", LpMaximize)
#     total_score += lpSum([Score[i] * player_vars[i] for i in player_vars])
#     total_score += lpSum([Cost[i] * player_vars[i] for i in player_vars]) <= AttSpend

#     MF = [p for p in positions.keys() if positions[p] == 3]
#     ST = [p for p in positions.keys() if positions[p] == 4]
#     CS = [p for p in positions.keys() if CurrentSelection[p] == 'N']
#     total_score += lpSum([player_vars[i] for i in MF]) == 5
#     total_score += lpSum([player_vars[i] for i in ST]) == 3
#     total_score += lpSum([player_vars[i] for i in CS]) == 8-FreeTransfers

#     total_score.solve()

#     for v in total_score.variables():
#         if v.varValue > 0:
#             print(v.name)


In [21]:
# def DefenceOptimisationproblem(Source,DefSpend,FreeTransfers):
#     Def_Elements_df=Source.dropna(subset=['DefenceOpp_pergame'])
#     Def_Elements_df['DefenceOpp_pergame'].astype(float)
#     Def_Elements_df=Def_Elements_df[Def_Elements_df['element_type_x']!=3]
#     Def_Elements_df=Def_Elements_df[Def_Elements_df['element_type_x']!=4]
#     ## Defence Variables
#     DefPlayers = list(Def_Elements_df['web_name'])
#     DefCost = dict(zip(DefPlayers, Def_Elements_df['now_cost']))
#     DefScore = dict(zip(DefPlayers, Def_Elements_df['DefenceOpp_pergame'].astype(float)))
#     Defpositions = dict(zip(DefPlayers, Def_Elements_df['element_type_x']))
#     DefCurrentSelection = dict(zip(DefPlayers, Def_Elements_df['IsInCurrentTeam']))

#     player_vars_def = LpVariable.dicts("DefPlayers", DefPlayers, lowBound=0, upBound=1, cat='Integer')

# ##Defence Constraints and problem solve

#     total_score_def = LpProblem("Fantasy_Points_Problem_Defence", LpMinimize)
#     total_score_def += lpSum([DefScore[i] * player_vars_def[i] for i in player_vars_def])
#     total_score_def += lpSum([DefCost[i] * player_vars_def[i] for i in player_vars_def]) <= DefSpend

#     GK = [p for p in Defpositions.keys() if Defpositions[p] == 1]
#     DF = [p for p in Defpositions.keys() if Defpositions[p] == 2]
#     #CS = [p for p in DefCurrentSelection.keys() if DefCurrentSelection[p] == 'N']
#     total_score_def += lpSum([player_vars_def[i] for i in GK]) == 2
#     total_score_def += lpSum([player_vars_def[i] for i in DF]) == 5
#     #total_score_def += lpSum([player_vars_def[i] for i in CS]) == 7-FreeTransfers

#     total_score_def.solve()

#     for v in total_score_def.variables():
#         if v.varValue > 0:
#             print(v.name)

In [ ]:
# AttackOptimisationproblem(Small_Elements_df,700,1)
# DefenceOptimisationproblem(Small_Elements_df,300,1)

In [ ]:
##### Returns Output 1 FixturesElementsAll
FixturesElementsAll1 = pd.merge(FixturesConcat, Elements_Dim2Grouped2,left_on='OppTeam2', right_on='OppTeam', how='outer')

FixturesElementsAll2 = pd.merge(grouped_dfLast4Weeks, FixturesElementsAll1,left_on='team', right_on='TeamID', how='outer')

FixturesElementsAll2['AttackOpp_pergame'] = FixturesElementsAll2['XGC_Weighted']*FixturesElementsAll2['expected_goal_involvements.gw']

FixturesElementsAll2['DefenceOpp_pergame'] = FixturesElementsAll2['XGI_Weighted']*FixturesElementsAll2['expected_goals_conceded.gw']

FixturesElementsAll2Att=FixturesElementsAll2[FixturesElementsAll2['element_type']>2]

# Calculate total AttackOpp_pergame for each player
FixturesElementsAll2Att = FixturesElementsAll2Att[FixturesElementsAll2Att['event']<UpcomingWeek+4]
FixturesElementsAll2Att.head(10)
FixturesElementsAll2_totalsAtt = FixturesElementsAll2Att.groupby('web_name')['AttackOpp_pergame'].sum().reset_index()

# Sort players by total AttackOpp_pergame and select top 20
top_players = FixturesElementsAll2_totalsAtt.sort_values(by='AttackOpp_pergame', ascending=False).head(10)

# Filter original DataFrame to include only top 20 players
FixturesElementsAll2AttFiltered = FixturesElementsAll2[FixturesElementsAll2['web_name'].isin(top_players['web_name'])]

FixturesElementsAll2AttFilteredslim = FixturesElementsAll2AttFiltered[['web_name','event','AttackOpp_pergame']]
FixturesElementsAll2AttFilteredslim['AttackOpp_pergame'].astype(float)
FixturesElementsAll2AttFilteredslim.sort_values(by='event', ascending=True)

FixturesElementsAll2AttFilteredslim=FixturesElementsAll2AttFilteredslim.drop_duplicates(subset=['event','web_name'])

figAttAtt = px.bar(data_frame=FixturesElementsAll2AttFilteredslim, x="event", y="AttackOpp_pergame", barmode='group', color='web_name')
